In [70]:
import pandas as pd
from google.cloud import storage
import os
from ydata_profiling import ProfileReport
from unidecode import unidecode

In [71]:
BUCKET_NAME = 'busca-apartamentos-trusted'
FILE = '2024-02-19 - apartamentos - cilar.csv'
TEMP_FILE = 'tmp/local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

files = [i.name for i in bucket.list_blobs()]

for file in files:
    blop = bucket.blob(blob_name= file).download_as_string()
    with open(f'tmp/{file}', 'wb') as f:
        f.write(blop)

In [72]:
df_full = pd.DataFrame()
for _, _, files_names in os.walk('tmp'):
    for file_name in files_names:
        df_aux = pd.read_csv('tmp/' + file_name)
        df_full = pd.concat([df_full, df_aux], axis = 0, ignore_index=True)

In [73]:
profile = ProfileReport(df_full, title="Profiling Report")
profile.to_file('EDA_full.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:  85%|████████▍ | 28/33 [00:03<00:00, 14.96it/s, Calculate auto correlation]              /home/arthur/anaconda3/envs/busca-apartamentos-env/lib/python3.11/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Não'')
  warnings.warn(
Summarize dataset:  93%|█████████▎| 67/72 [00:16<00:02,  1.81it/s, Missing diagram heatmap]                 /home/arthur/anaconda3/envs/busca-apartamentos-env/lib/python3.11/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:"

In [74]:
df_full[['aluguel','condominio','seguro_incendio','iptu']] = df_full[['aluguel','condominio','seguro_incendio','iptu']].fillna(0)

In [75]:
df_full['valor_total'] = df_full['aluguel'] + df_full['condominio'] + df_full['seguro_incendio'] + df_full['iptu']

In [76]:
unidecode(df_full.loc[0,'bairro'].lower())

'agua verde'

In [77]:
df_full['bairro'] = df_full['bairro'].apply(lambda x:x  if isinstance(x,float) else unidecode(x.lower().strip()))

In [78]:
features = [
 'link',
 'bairro',
 'cidade',
 'aluguel',
 'condominio',
 'seguro_incendio',
 'iptu',
 'area',
 'quartos',
 'suites',
 'banheiros',
 'vagas_garagem',
 'mobiliado',
 'piscina',
 'academia',
 'sacada',
 'churrasqueira',
 'salao_de_festas',
 'valor_total']

In [79]:
df = df_full[features]

In [ ]:
df[['bairro','valor_total']].groupby('bairro').mean().sort_values('valor_total', ascending= False).head(30)

In [82]:
df[['bairro','valor_total']].groupby('bairro').count().sort_values('valor_total', ascending= False).head(30)

,valor_total
bairro,
centro,159
agua verde,55
portao,45
bigorrilho,43
batel,36
cristo rei,34
vila izabel,23
alto da gloria,22
reboucas,21


In [80]:
df[['bairro','valor_total']].groupby('bairro').mean('valor_total').sort_values('valor_total', ascending= False).head(30)

,valor_total
bairro,
ecoville,6704.526471
guabirotuba,5674.750000
mossungue,5337.315000
merces,4818.641250
seminario,4622.110000
juveve,4464.920714
cabral,4130.963158
batel,4091.658889
agua verde,3949.135091
